# <img src="icons\meteo2maps.png" width=20 height=20/> METEO2MAPS
## <img src="icons\meteo2maps.png" width=20 height=20/> Projeto prático de software aberto e programação em SIG
### <img src="icons\meteo2maps.png" width=20 height=20/> João H. Oliveira, 2021

In [75]:
import os, urllib.request, json, subprocess, geopandas, pandas, shutil, psycopg2, re, requests, time
from datetime import datetime
from psycopg2 import extras as psy2extras
from geo.Geoserver import Geoserver

In [76]:
os.environ['SHAPE_ENCODING'] = "utf-8"
basePath = 'C:\\saprog\\projeto'
outputPath = 'C:\\saprog\\projeto\\output'
os.chdir(basePath)

def deleteAndCreateOutput():
    print ('A limpar e criar directoria de outputs...')
    outDir = os.path.join(basePath, 'output')
    if os.path.isdir(outDir):
        shutil.rmtree(outDir)
    os.mkdir(outDir)

deleteAndCreateOutput()

A limpar e criar directoria de outputs...


In [77]:
print('A ler shapefile...')
caop = geopandas.read_file('caop.shp', encoding='utf-8')
# caop.plot()
# caop.head()
# gdf = caop.loc[caop['Concelho'] == 'Bombarral']
# gdf.head()

A ler shapefile...


In [78]:
os.chdir(outputPath)

print('A executar Dissolve...')
distritos = caop.dissolve(by = 'Distrito')

print('A exportar Distritos...')
distritos.to_file('distritos.shp', encoding='utf-8')

# distritos.plot()
# distritos.head(1)

A executar Dissolve...
A exportar Distritos...


In [72]:
print('A extrair centroides das regiões admninistrativas...')

## Para evitar o warning da reprojeção
# dataProj = distritos.to_crs(epsg = 3763)
# centroides = dataProj.centroid

os.chdir(outputPath)

centroides = distritos.centroid
centroides.to_file('centroides.shp', encoding='utf-8')
# centroides.plot()
# centroides.head()

coordx = centroides.x.to_dict()
coordy = centroides.y.to_dict()
# print(coordx)
# print(coordy)

A extrair centroides das regiões admninistrativas...
<ipython-input-72-507fe550f694>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroides = distritos.centroid


In [79]:
def getCoordinates(dicX, dicY):
    '''
    Extração de coordenadas geográficas úteis ao harvest de dados meteorológicos.
    Devolve um dicionário onde as chaves são os distritos, à qual está associado um tuplo com as               coordenadas Lat Long.
    '''

    coord = [dicX, dicY]
    coordDic = {}
    for i in dicX.keys():
        coordDic[i] = tuple(coordDic[i] for coordDic in coord)
    return coordDic

print('A extrair coodenadas geográficas...')
coord = getCoordinates(coordx, coordy)
# print(coord)

A extrair coodenadas geográficas...


In [73]:
# Carregamento de shapefile "distritos" na bd
command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
          "-f", "PostgreSQL",
          "PG:host=localhost user=postgres dbname=meteo password=3763", outputPath,
          "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
          " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "distritos", "-overwrite"]
subprocess.check_call(command)

# Carregamento de shapefile "centroides" na bd
command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
          "-f", "PostgreSQL",
          "PG:host=localhost user=postgres dbname=meteo password=3763", outputPath,
          "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
          " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "centroides", "-overwrite"]
subprocess.check_call(command)

0

In [80]:
apiKey = '44cfad7f82ec61d3f420de3201b703d4'

def getWeather(coordDic, apikey):
    '''
    Função para recolha dos dados meteorológicos provenientes da OpenWeather One Call API.
    Devolve uma dataframe das váriáveis meteorológicas por distrito.
    '''
    forecast = []
    for item in coord.items():
        lat = str(item[1][1])
        long = str(item[1][0])
        url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + long + \
            '&exclude=minutely,hourly,daily,alerts&appid=' + apiKey + '&units=metric'
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
            districtForecast = {}
            districtForecast['distrito'] = item[0]
            districtForecast['forecast_date'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%d-%m-%Y')
            districtForecast['forecast_time'] = datetime.utcfromtimestamp(data.get('current').get('dt')).strftime('%H:%M:%S')
            main = data.get('current').get('weather')
            for item in main:
                districtForecast['weather_desc'] = item.get('main')
            districtForecast['temperature'] = data.get('current').get('temp')
            districtForecast['feels_like'] = data.get('current').get('feels_like')
            districtForecast['pressure'] = data.get('current').get('pressure')
            districtForecast['humidity'] = data.get('current').get('humidity')
            districtForecast['dew_point'] = data.get('current').get('dew_point')
            districtForecast['ultrav_index'] = data.get('current').get('uvi')
            districtForecast['wind_speed'] = data.get('current').get('wind_speed')
            districtForecast['wind_deg'] = data.get('current').get('wind_deg')
            forecast.append(districtForecast)
    forecast_df = pandas.DataFrame(forecast)
    return forecast_df

forecast_df = getWeather(coord, apiKey)
forecast_df.head(-1)

,distrito,forecast_date,forecast_time,weather_desc,temperature,feels_like,pressure,humidity,dew_point,ultrav_index,wind_speed,wind_deg
0,Aveiro,26-03-2021,00:25:51,Clouds,11.11,9.55,1025,83,8.34,0,1.68,29
1,Beja,26-03-2021,00:25:52,Clear,11.00,8.75,1023,82,8.05,0,2.57,290
2,Braga,26-03-2021,00:25:52,Clouds,8.48,7.23,1023,89,6.77,0,0.72,60
3,Bragança,26-03-2021,00:25:52,Clouds,8.44,6.30,1022,75,4.27,0,1.24,274
4,Castelo Branco,26-03-2021,00:25:53,Clouds,6.53,4.58,1014,91,5.17,0,1.23,307
5,Coimbra,26-03-2021,00:25:53,Clouds,10.35,9.84,1021,89,8.62,0,0.28,332
6,Faro,26-03-2021,00:25:54,Clear,14.00,13.31,1023,88,12.05,0,1.89,17
7,Guarda,26-03-2021,00:25:54,Clouds,5.36,2.58,1023,83,2.71,0,1.75,256
8,Leiria,26-03-2021,00:25:54,Clouds,7.88,6.25,1025,94,6.98,0,1.32,10
9,Lisboa,26-03-2021,00:25:55,Clouds,10.72,9.36,1023,88,8.82,0,1.56,355


In [82]:
os.chdir(basePath)

password = open(os.path.join('pw.txt'), 'r').readline()
password = str(password)

con = psycopg2.connect(dbname='meteo', user='postgres', password=password, host='localhost', port='5432')

def df2PgSQL(conn, df, table):
    '''
    Utilização da função psycopg2.extras.execute_values()
    para carregamento da data frame colhida na tabela PostGreSQL "forecast"
    '''
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        psy2extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("dfToPgSQL() done")
    cursor.close()

df2PgSQL(con, forecast_df, 'forecast')

dfToPgSQL() done


In [74]:
# # Carregamento de shapefile "caop" na bd
# command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
#           "-f", "PostgreSQL",
#           "PG:host=localhost user=postgres dbname=meteo password=3763",
#           outputPath,
#           "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
#           " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", "caop", "-overwrite"]

# subprocess.check_call(command)

# pgConnectionPar = {'host':'localhost', 'user':'postgres', 'dbname':'meteo', 'password':'3763'}

# def shp2PgSQL(shp, pgconn):
#     command = ["C:\\OSGeo4W64\\bin\\ogr2ogr.exe",
#             "-f", "PostgreSQL",
#             "PG:host={} user={} dbname={} password={}".format(pgconn.get('host'),\
#             pgconn.get('user'), pgconn.get('dbname'), pgconn.get('password')),
#             outputPath,
#             "-lco", "GEOMETRY_NAME=the_geom", "-lco", "FID=gid", "-lco",
#             " PRECISION=no", "-nlt", "PROMOTE_TO_MULTI", "-nln", shp+'.shp', "-overwrite"]
#     exe = subprocess.Popen(command,stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
#     for line in exe.stdout:
#         line = str(line.decode('utf-8'))
#         line_txt = re.sub("\r?\n", '', line)
#         print(line_txt)

# shp2PgSQL('caop', pgConnectionPar)


In [ ]:
# falta criar código para produção de view dentro da base de dados + join com caop poligonal

drop view if exists last_forecast;
create view last_forecast as
select forecast.*, centroides.the_geom
from forecast, centroides
where forecast.distrito = centroides.distrito
order by forecast.forecast_id desc limit 18

In [41]:
# Start Geoerver with start "" /D C:\"Program Files"\geoserver\bin\ /W startup.bat

# Initialize the library
geo = Geoserver('http://localhost:8080/geoserver', username='admin', password='geojoao')

# # For creating workspace
geo.create_workspace(workspace='sapsig_meteo')

'201 Workspace sapsig_meteo created!'